In [81]:
# https://github.com/awjuliani/DeepRL-Agents/blob/master/Contextual-Policy.ipynb

In [82]:
import keras
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Activation
from keras.optimizers import Adam, SGD
from keras import metrics
from keras import backend as K
import numpy as np

## The Contextual Bandits

In [254]:
class contextualBandit:
    def __init__(self):
        self.state = 0
        #List out our bandits. Currently arms 4, 2, and 1 (respectively) are the most optimal.
        self.bandits = np.array([[0.2,0,0.0,-5],[0.1,-5,1,0.25],[-5,5,5,5]])
        self.num_bandits = self.bandits.shape[0]
        self.num_actions = self.bandits.shape[1]
        
    def getBandit(self):
        self.state = np.random.randint(0,len(self.bandits)) #Returns a random state for each episode.
        return self.state
        
    def pullArm(self,action):
        #Get a random number.
        bandit = self.bandits[self.state,action]
        result = np.random.randn(1)
        if result > bandit:
            #return a positive reward.
            return 1
        else:
            #return a negative reward.
            return -1

## The Policy-Based Agent

In [384]:
def getAgentModel(input_size, output_size):
    ii = Input(shape=(input_size,))
    oo = Dense(8, activation='relu')(ii)
    oo = Dense(output_size, activation='softmax')(oo)
    model = Model(ii, oo)
    return model

In [385]:
agent = getAgentModel(3, 4)
agent.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_19 (InputLayer)        (None, 3)                 0         
_________________________________________________________________
dense_29 (Dense)             (None, 8)                 32        
_________________________________________________________________
dense_30 (Dense)             (None, 4)                 36        
Total params: 68
Trainable params: 68
Non-trainable params: 0
_________________________________________________________________


In [386]:
def loss_custom(y_true,y_pred):
    # self.loss = -(tf.log(self.responsible_weight)*self.reward_holder)
    return -(K.log(y_pred)*y_true)

In [387]:
# my_loss = 'mean_squared_error'
# my_loss = "binary_crossentropy"
my_loss = loss_custom

opt = keras.optimizers.Adam(lr=0.001)
# opt = keras.optimizers.SGD(lr=0.001)
agent.compile(optimizer=opt, loss=my_loss)

## Training the Agent

In [388]:
cBandit = contextualBandit()

In [392]:
total_episodes = 5000 #Set total number of episodes to train agent on.
total_reward = np.zeros([cBandit.num_bandits,cBandit.num_actions]) #Set scoreboard for bandits to 0.
e = 0.1 #Set the chance of taking a random action.

In [393]:
def get_one_hot(idx, size, val=1, def_vals=0):
    oh = np.full(size, def_vals)
    oh[idx] = val
    return oh
# UNIT TEST
print(get_one_hot(5, 10))

[0 0 0 0 0 1 0 0 0 0]


In [394]:
def ll(y_true,y_pred):
    return -(np.log(y_pred)*y_true)
print(ll(1, 0.32342818))
print(ll(1, 0.2776792))
print(ll(-1, 0.32342818))
print(ll(-1, 0.2776792))

1.128778198962459
1.2812887884468733
-1.128778198962459
-1.2812887884468733


In [397]:
i = 0
DBG = 0
while i < total_episodes:
    s = cBandit.getBandit() #Get a state from the environment.
    if DBG:
        print("\nstate(s) : {}".format(s))
    
    #Choose either a random action or one from our network.
    if np.random.rand(1) < e:
        action = np.random.randint(cBandit.num_actions)
        if DBG:
            print("action raw (random) : {}".format(action))
    else:
        action = agent.predict(np.array([get_one_hot(s, cBandit.num_bandits)]))
        if DBG:
            print("action raw (agent) : {}".format(action))
    action_val = np.argmax(action)
    if DBG:
        print ("action_val : {}".format(action_val))
    
    #Get our reward for taking an action given a bandit.
    reward = cBandit.pullArm(action_val)
    if DBG:
        print("reward : {}".format(reward))
    
    #Update the network.
    state_arr = get_one_hot(s, cBandit.num_bandits)
    reward_arr = get_one_hot(action_val, cBandit.num_actions, val=reward, def_vals=0)
    #reward_arr = np.full(cBandit.num_actions, reward)
    if DBG:
        print("X : state = {}, Y : reward : {}".format(state_arr, reward_arr))
    agent.fit(np.array([state_arr]), np.array([reward_arr]),
                epochs=1,
                batch_size=1
                # shuffle=True
                # validation_data=(x_test, x_test)
               )
    i += 1

Epoch 1/1
1/1 [==============================] - 0s 998us/step - loss: 0.0012
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.9303
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.0012
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.0234
Epoch 1/1
1/1 [==============================] - 0s 3ms/step - loss: 0.0235
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.0235
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.0012
Epoch 1/1
1/1 [==============================] - 0s 4ms/step - loss: 0.0012
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.0236
Epoch 1/1
1/1 [==============================] - 0s 3ms/step - loss: 0.0012
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.0012
Epoch 1/1
1/1 [==============================] - 0s 997us/step - loss: 0.1878
Epoch 1/1
1/1 [==============================] - 0s 999us/step - loss: 0.0237
Epoch 

Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.0011
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.0011
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.1769
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.0011
Epoch 1/1
1/1 [==============================] - 0s 997us/step - loss: 0.1762
Epoch 1/1
1/1 [==============================] - 0s 3ms/step - loss: -0.1758
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: -0.1756
Epoch 1/1
1/1 [==============================] - 0s 998us/step - loss: -1.0030
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.0172
Epoch 1/1
1/1 [==============================] - 0s 3ms/step - loss: -0.1750
Epoch 1/1
1/1 [==============================] - 0s 997us/step - loss: 0.1749
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.0169
Epoch 1/1
1/1 [==============================] - 0s 997us/step - loss: 0.0011


Epoch 1/1
1/1 [==============================] - 0s 997us/step - loss: -0.1702
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.0010
Epoch 1/1
1/1 [==============================] - 0s 997us/step - loss: 0.0010
Epoch 1/1
1/1 [==============================] - 0s 5ms/step - loss: -1.1132
Epoch 1/1
1/1 [==============================] - 0s 3ms/step - loss: 0.1712
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.0010
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.1708
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.0010
Epoch 1/1
1/1 [==============================] - 0s 998us/step - loss: 0.0010
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.0010
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.1696
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.1692
Epoch 1/1
1/1 [==============================] - 0s 3ms/step - loss: 0.0119
Epoc

Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.1831
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 8.8178e-04
Epoch 1/1
1/1 [==============================] - 0s 997us/step - loss: 0.0129
Epoch 1/1
1/1 [==============================] - 0s 998us/step - loss: 8.7804e-04
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.1838
Epoch 1/1
1/1 [==============================] - 0s 3ms/step - loss: 0.0130
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 8.7391e-04
Epoch 1/1
1/1 [==============================] - 0s 997us/step - loss: 8.7267e-04
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: -0.1834
Epoch 1/1
1/1 [==============================] - 0s 997us/step - loss: 1.0791
Epoch 1/1
1/1 [==============================] - 0s 3ms/step - loss: 8.6675e-04
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.0132
Epoch 1/1
1/1 [==============================] - 0s 996us/s

1/1 [==============================] - 0s 3ms/step - loss: 0.1850
Epoch 1/1
1/1 [==============================] - 0s 997us/step - loss: 0.0134
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: -0.1848
Epoch 1/1
1/1 [==============================] - 0s 997us/step - loss: 0.0133
Epoch 1/1
1/1 [==============================] - 0s 3ms/step - loss: 0.0133
Epoch 1/1
1/1 [==============================] - 0s 998us/step - loss: 7.8505e-04
Epoch 1/1
1/1 [==============================] - 0s 998us/step - loss: 0.0133
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 7.8346e-04
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: -0.1852
Epoch 1/1
1/1 [==============================] - 0s 999us/step - loss: 7.8153e-04
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.0133
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: -0.1858
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.01

...

Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 8.2715e-05
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 8.2715e-05
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.0767
Epoch 1/1
1/1 [==============================] - 0s 4ms/step - loss: 0.0017
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.0763
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.0017
Epoch 1/1
1/1 [==============================] - 0s 4ms/step - loss: 0.0760
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: -0.6755
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 8.2834e-05
Epoch 1/1
1/1 [==============================] - 0s 3ms/step - loss: 0.0751
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 8.2983e-05
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.0746
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.

Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 7.9972e-05
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.0014
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.0641
Epoch 1/1
1/1 [==============================] - 0s 997us/step - loss: 0.0014
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.0014
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.0639
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 7.9555e-05
Epoch 1/1
1/1 [==============================] - 0s 3ms/step - loss: 0.0014
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 7.9466e-05
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 7.9376e-05
Epoch 1/1
1/1 [==============================] - 0s 4ms/step - loss: 0.0014
Epoch 1/1
1/1 [==============================] - 0s 3ms/step - loss: -1.9109
Epoch 1/1
1/1 [==============================] - 0s 997us/step - loss

Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 7.5232e-05
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.0012
Epoch 1/1
1/1 [==============================] - 0s 3ms/step - loss: 7.5083e-05
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.0012
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.0012
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.0550
Epoch 1/1
1/1 [==============================] - 0s 998us/step - loss: 0.0550
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.0549
Epoch 1/1
1/1 [==============================] - 0s 998us/step - loss: 0.0012
Epoch 1/1
1/1 [==============================] - 0s 3ms/step - loss: 0.0012
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 7.4621e-05
Epoch 1/1
1/1 [==============================] - 0s 997us/step - loss: 0.0012
Epoch 1/1
1/1 [==============================] - 0s 996us/step - loss:

Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.0486
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.0012
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.7926
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.0485
Epoch 1/1
1/1 [==============================] - 0s 998us/step - loss: 6.8450e-05
Epoch 1/1
1/1 [==============================] - 0s 998us/step - loss: 0.0012
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.0485
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 6.8182e-05
Epoch 1/1
1/1 [==============================] - 0s 998us/step - loss: 0.0012
Epoch 1/1
1/1 [==============================] - 0s 998us/step - loss: 6.8003e-05
Epoch 1/1
1/1 [==============================] - 0s 997us/step - loss: 0.0012
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.0012
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - los

1/1 [==============================] - 0s 2ms/step - loss: 0.0011
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.0011
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 6.1833e-05
Epoch 1/1
1/1 [==============================] - 0s 3ms/step - loss: 6.1773e-05
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.0011
Epoch 1/1
1/1 [==============================] - 0s 3ms/step - loss: 6.1654e-05
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 6.1564e-05
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 6.1475e-05
Epoch 1/1
1/1 [==============================] - 0s 4ms/step - loss: 0.0426
Epoch 1/1
1/1 [==============================] - 0s 998us/step - loss: 0.0011
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.0425
Epoch 1/1
1/1 [==============================] - 0s 997us/step - loss: 0.0011
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 6.111

In [398]:
print("CHECK AGENT PREDICTION")
res0 = agent.predict(np.array([[1, 0, 0]]))
print("agent predict 0 bandit's best : {} => {}".format(res0, np.argmax(res0)))
res1 = agent.predict(np.array([[0, 1, 0]]))
print("agent predict 1 bandit's best : {} => {}".format(res1, np.argmax(res1)))
res2 = agent.predict(np.array([[0, 0, 1]]))
print("agent predict 2 bandit's best : {} => {}".format(res2, np.argmax(res2)))

CHECK AGENT PREDICTION
agent predict 0 bandit's best : [[4.0332368e-04 3.6346521e-03 4.2193232e-04 9.9554008e-01]] => 3
agent predict 1 bandit's best : [[0.03735428 0.8446924  0.04510457 0.07284874]] => 1
agent predict 2 bandit's best : [[9.99761403e-01 8.88754221e-05 4.89004342e-05 1.00747566e-04]] => 0


In [267]:
print("THE ANSWER IS")
print("0 bandit : {} => {} is best".format(cBandit.bandits[0], np.argmin(cBandit.bandits[0])))
print("1 bandit : {} => {} is best".format(cBandit.bandits[1], np.argmin(cBandit.bandits[1])))
print("2 bandit : {} => {} is best".format(cBandit.bandits[2], np.argmin(cBandit.bandits[2])))

THE ANSWER IS
0 bandit : [ 0.2  0.   0.  -5. ] => 3 is best
1 bandit : [ 0.1  -5.    1.    0.25] => 1 is best
2 bandit : [-5.  5.  5.  5.] => 0 is best
